# Flow Matching - Coding Challenge

## What is Flow Matching?

Flow Matching is a **generative model** that transforms random noise into structured data by solving an ODE:

```
dx/dt = v(x, t)   for t ∈ [0, 1]
```

| Time | State | Description |
|------|-------|-------------|
| t = 0 | Source | Random Gaussian noise |
| t = 1 | Target | Structured data (clusters) |
| v(x,t) | Velocity | Guides toward nearest cluster |

---

## ⚠️ This is a Simplified Demo (Not Real Flow Matching)

| | Real Flow Matching | This Demo |
|---|---|---|
| **Velocity v(x,t)** | Learned by neural network | Hand-crafted formula |
| **Training** | Trained on data | No training needed |
| **Formula** | `v = network(x, t)` | `v = (target - x) / (1 - t)` |

**What's real here:**
- ✓ The ODE concept: `dx/dt = v(x, t)`
- ✓ The ODE solvers (Euler, RK4)
- ✓ Flow from noise → structured data

**What's simplified:**
- ✗ No neural network
- ✗ No training
- ✗ Velocity field is hand-crafted

---

## This Notebook Demonstrates

| | 2D Version | Audio Version |
|---|------------|---------------|
| **Source** | Random points | White noise |
| **Target** | 4 spatial clusters | 4 frequency clusters |
| **Clusters** | (3,0), (0,3), (-3,0), (0,-3) | C4, E4, G4, C5 (262-523 Hz) |
| **Flow** | Points → nearest cluster | Noise → nearest frequency |

---

## Notebook Structure

### Setup (Run First)
- Imports, model definitions, ODE solvers

### Loom Video Recording 1: Flow Matching Concept (2 min)
**Key questions to answer:**
- What is the source distribution? → Gaussian noise at t=0
- What is the target distribution? → 4 clusters at t=1
- What does the velocity field do? → Points toward nearest cluster
- Why do trajectories curve? → Velocity increases as t→1

### Loom Video Recording 2: ODE Solvers Comparison (2 min)
**Key questions to answer:**
- Which solver is fastest? → Euler (1 eval/step)
- Which solver is most accurate? → RK4 (4 evals/step)
- What is the trade-off? → Speed vs Accuracy
- What happens with stiff ODEs? → Euler becomes unstable

---

## Setup: Run This First
Run all cells in this section before recording any video.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Optional
from dataclasses import dataclass
from IPython.display import Audio, display

%matplotlib inline

# ============================================
# AUDIO FLOW MATCHING WITH REAL ODE SOLVERS
# ============================================
SAMPLE_RATE = 22050

def generate_noise(duration=1.0):
    """Source: Random noise audio."""
    n_samples = int(SAMPLE_RATE * duration)
    return np.random.randn(n_samples) * 0.3

def generate_target_tone(frequency=440, duration=1.0):
    """Target: Clean sine wave tone."""
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), False)
    return np.sin(2 * np.pi * frequency * t) * 0.3

class AudioFlowModel:
    """Audio flow matching model using ODE."""
    
    def __init__(self, target_audio):
        self.target = target_audio
    
    def get_velocity(self, x, t):
        """
        Velocity field for audio: v(x,t) = (target - x) / (1 - t + eps)
        Same formula as 2D flow matching!
        """
        return (self.target - x) / (1 - t + 0.01)

def audio_euler_step(x, t, dt, velocity_fn):
    """Euler method for audio."""
    return x + dt * velocity_fn(x, t)

def audio_rk4_step(x, t, dt, velocity_fn):
    """RK4 method for audio."""
    k1 = velocity_fn(x, t)
    k2 = velocity_fn(x + (dt/2) * k1, t + dt/2)
    k3 = velocity_fn(x + (dt/2) * k2, t + dt/2)
    k4 = velocity_fn(x + dt * k3, t + dt)
    return x + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)

def generate_audio_flow(source, target, step_fn, num_steps=50, t_end=1.0):
    """
    Generate audio using ODE flow matching.
    
    Args:
        source: Starting audio (noise)
        target: Target audio (tone)
        step_fn: ODE solver (euler, rk4)
        num_steps: Number of integration steps
        t_end: Stop integration at this t value (0 to 1)
    """
    model = AudioFlowModel(target)
    dt = t_end / num_steps
    x = source.copy()
    
    for i in range(num_steps):
        t = i * dt
        x = step_fn(x, t, dt, model.get_velocity)
    
    return x

def play(audio, rate=SAMPLE_RATE):
    """Play audio in notebook."""
    display(Audio(audio, rate=rate, autoplay=False))

print("Audio Flow Matching with ODE ready!")
print("- Uses same velocity field formula as 2D: v = (target - x) / (1 - t)")
print("- Supports Euler, RK4 solvers")
print("- t_end controls how far the flow goes (0=noise, 1=tone)")

In [ ]:
@dataclass
class FlowConfig:
    dim: int = 2
    num_targets: int = 4
    seed: Optional[int] = None


class FlowMatchingModel:
    """Mock flow matching model with Gaussian mixture target."""
    
    def __init__(self, config: FlowConfig):
        self.config = config
        self.rng = np.random.default_rng(config.seed)
        
        # Target: Gaussian clusters arranged in a circle
        angles = np.linspace(0, 2 * np.pi, config.num_targets, endpoint=False)
        self.centers = 3.0 * np.stack([np.cos(angles), np.sin(angles)], axis=1)
    
    def sample_source(self, n: int) -> np.ndarray:
        """Sample from source distribution (standard Gaussian)."""
        return self.rng.standard_normal((n, self.config.dim))
    
    def sample_target(self, n: int) -> np.ndarray:
        """Sample from target distribution (Gaussian mixture)."""
        indices = self.rng.integers(0, len(self.centers), n)
        samples = np.array([
            self.rng.multivariate_normal(self.centers[i], np.eye(2) * 0.3)
            for i in indices
        ])
        return samples
    
    def get_velocity(self, x: np.ndarray, t: float) -> np.ndarray:
        """Compute velocity field v(x, t)."""
        squeeze = x.ndim == 1
        x = np.atleast_2d(x)
        batch_size = x.shape[0]
        
        velocities = np.zeros_like(x)
        for i in range(batch_size):
            dists = np.linalg.norm(self.centers - x[i], axis=1)
            weights = np.exp(-dists / (1 + t))
            weights /= weights.sum()
            target = np.average(self.centers, axis=0, weights=weights)
            velocities[i] = (target - x[i]) / (1 - t + 0.01)
        
        return velocities.squeeze() if squeeze else velocities

In [ ]:
# ODE Solvers
def euler_step(x, t, dt, velocity_fn):
    """Euler: 1st order, 1 eval/step"""
    return x + dt * velocity_fn(x, t)

def rk4_step(x, t, dt, velocity_fn):
    """RK4: 4th order, 4 eval/step"""
    k1 = velocity_fn(x, t)
    k2 = velocity_fn(x + (dt/2) * k1, t + dt/2)
    k3 = velocity_fn(x + (dt/2) * k2, t + dt/2)
    k4 = velocity_fn(x + dt * k3, t + dt)
    return x + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)

In [ ]:
def generate_samples(model, step_fn, n_samples=100, num_steps=50, return_trajectories=False):
    """Generate samples by integrating flow from t=0 to t=1."""
    dt = 1.0 / num_steps
    x = model.sample_source(n_samples)
    trajectories = [x.copy()] if return_trajectories else None
    
    for i in range(num_steps):
        t = i * dt
        x = step_fn(x, t, dt, model.get_velocity)
        if return_trajectories:
            trajectories.append(x.copy())
    
    if return_trajectories:
        return x, np.array(trajectories)
    return x

In [ ]:
# Initialize model
config = FlowConfig(dim=2, num_targets=4, seed=42)
model = FlowMatchingModel(config)

print("=" * 60)
print("FLOW MATCHING DEMONSTRATION")
print("=" * 60)

print("\n[1] MODEL SETUP")
print("-" * 40)
print(f"Dimension: {config.dim}")
print(f"Number of target clusters: {config.num_targets}")
print(f"Cluster centers:")
print(model.centers)

print("\nSetup complete!")

---
# Loom video recording 1: Flow Matching Concept (2 min)

**Key points to explain:**
1. What is source distribution (Gaussian noise at t=0)
2. What is target distribution (4 clusters at t=1)
3. How the velocity field guides points from noise to clusters
4. Why trajectories curve toward cluster centers

---

In [ ]:
# 🔊 AUDIO FLOW MATCHING WITH 4 FREQUENCY CLUSTERS - Loom video recording 1
print("AUDIO FLOW MATCHING with 4 Frequency Clusters")
print("=" * 50)
print("Just like 2D has 4 spatial clusters, audio has 4 frequency clusters!\n")

# 4 frequency clusters (like 4 spatial clusters in 2D)
FREQ_CLUSTERS = [262, 330, 392, 523]  # C4, E4, G4, C5 (C major chord)
FREQ_NAMES = ["C4 (262 Hz)", "E4 (330 Hz)", "G4 (392 Hz)", "C5 (523 Hz)"]

print("Frequency Clusters (like spatial clusters in 2D):")
for name in FREQ_NAMES:
    print(f"  • {name}")

# ============================================
# Audio Flow Model with Multiple Clusters
# ============================================
class MultiClusterAudioFlow:
    """Audio flow matching with multiple frequency clusters."""
    
    def __init__(self, frequencies, duration=1.0):
        self.frequencies = frequencies
        self.duration = duration
        # Generate target tones for each cluster
        t = np.linspace(0, duration, int(SAMPLE_RATE * duration), False)
        self.targets = [np.sin(2 * np.pi * f * t) * 0.3 for f in frequencies]
        self.centers = np.array(frequencies)  # For distance calculation
    
    def get_velocity(self, x, t_flow, initial_noise):
        """
        Velocity field that pulls toward nearest frequency cluster.
        Uses initial noise value to determine cluster assignment (like 2D).
        """
        # Determine which cluster based on initial noise characteristics
        # Map noise mean to cluster (similar to 2D spatial position → cluster)
        noise_feature = np.mean(initial_noise[:1000])  # Use first part as "position"
        
        # Soft assignment to clusters (like 2D weighted average)
        cluster_positions = np.linspace(-0.3, 0.3, len(self.frequencies))
        dists = np.abs(cluster_positions - noise_feature)
        weights = np.exp(-dists * 10)
        weights /= weights.sum()
        
        # Weighted target (mixture of frequency targets)
        target = sum(w * t for w, t in zip(weights, self.targets))
        
        return (target - x) / (1 - t_flow + 0.01)

def generate_multicluster_audio_flow(source, model, num_steps=30, t_end=1.0):
    """Generate audio flowing toward nearest frequency cluster."""
    dt = t_end / num_steps
    x = source.copy()
    initial_noise = source.copy()  # Remember starting point for cluster assignment
    
    for i in range(num_steps):
        t = i * dt
        v = model.get_velocity(x, t, initial_noise)
        x = x + dt * v
    
    return x

# ============================================
# Demo: 4 different noise samples → 4 different frequency clusters
# ============================================
print("\n" + "=" * 50)
print("4 NOISE SAMPLES → 4 FREQUENCY CLUSTERS")
print("=" * 50)
print("Each noise sample flows to its nearest frequency cluster")
print("(like 2D points flowing to nearest spatial cluster)\n")

duration = 1.0
model_multi = MultiClusterAudioFlow(FREQ_CLUSTERS, duration)

# Generate 4 noise samples with different characteristics
np.random.seed(42)
noise_samples = []
for i in range(4):
    # Shift mean slightly to target different clusters
    noise = np.random.randn(int(SAMPLE_RATE * duration)) * 0.3
    noise[:1000] += (i - 1.5) * 0.2  # Different "positions" in noise space
    noise_samples.append(noise)

print("▶ Starting noises (4 different 'positions'):")
for i in range(4):
    mean_val = np.mean(noise_samples[i][:1000])
    print(f"  Noise {i+1}: mean = {mean_val:+.3f}")
    play(noise_samples[i])

print("\n" + "-" * 50)
print("▶ After flow (each → nearest frequency cluster):")
print("-" * 50)

results = []
for i in range(4):
    result = generate_multicluster_audio_flow(noise_samples[i], model_multi, num_steps=30, t_end=1.0)
    results.append(result)
    print(f"\n  Noise {i+1} → {FREQ_NAMES[i]}:")
    play(result)

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 6))

for i in range(4):
    # Top row: noise (source)
    axes[0, i].plot(noise_samples[i][:500], 'b', linewidth=0.5)
    axes[0, i].set_title(f"Noise {i+1}\n(source)", fontsize=10)
    axes[0, i].set_ylim(-0.6, 0.6)
    axes[0, i].grid(True, alpha=0.3)
    
    # Bottom row: result (after flow)
    axes[1, i].plot(results[i][:500], 'g', linewidth=0.5)
    axes[1, i].set_title(f"→ {FREQ_NAMES[i]}\n(target cluster)", fontsize=10)
    axes[1, i].set_ylim(-0.6, 0.6)
    axes[1, i].grid(True, alpha=0.3)

axes[0, 0].set_ylabel("SOURCE\n(noise)", fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel("TARGET\n(tone)", fontsize=11, fontweight='bold')

plt.suptitle("Audio Flow Matching: 4 Noises → 4 Frequency Clusters\n(Analogous to 2D points → 4 spatial clusters)", fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n→ Just like 2D: different starting positions → different target clusters!")
print("→ Noise 1 → C4, Noise 2 → E4, Noise 3 → G4, Noise 4 → C5")

In [ ]:
# 1.2 Explain SOURCE distribution (Gaussian noise)
print("SOURCE DISTRIBUTION (t=0): Standard Gaussian Noise")
print("-" * 50)
source_samples = model.sample_source(5)
print("Sample points from source (random noise):")
print(source_samples)
print("\n→ These are random points centered at origin")

In [ ]:
# 1.3 Explain TARGET distribution (4 clusters)
print("TARGET DISTRIBUTION (t=1): 4 Gaussian Clusters")
print("-" * 50)
target_samples = model.sample_target(5)
print("Sample points from target (clustered):")
print(target_samples)
print("\n→ These points are grouped around the 4 cluster centers")

In [ ]:
# 1.4 Explain VELOCITY FIELD - what it does and why trajectories curve
print("VELOCITY FIELD: v(x, t) = (target - x) / (1 - t)")
print("-" * 50)
print("Testing velocity at different positions and times:\n")

test_positions = [np.array([0., 0.]), np.array([2., 0.]), np.array([0., 2.])]
test_times = [0.0, 0.5, 0.9]

for t in test_times:
    print(f"  Time t = {t}:")
    for x in test_positions:
        v = model.get_velocity(x, t)
        print(f"    x = {x} → v = [{v[0]:+.3f}, {v[1]:+.3f}]")
    print()

print("→ Velocity points TOWARD nearest cluster center")
print("→ As t→1, velocity increases (1-t shrinks)")
print("→ This is why trajectories CURVE toward clusters!")

In [ ]:
# 1.4 Show the TRAJECTORIES plot - how points flow from noise to clusters
print("FLOW TRAJECTORIES: How points move from noise → clusters")
print("-" * 50)

model.rng = np.random.default_rng(123)
samples, trajectories = generate_samples(
    model, rk4_step, n_samples=50, num_steps=40, return_trajectories=True
)

# Plot trajectories
fig, ax = plt.subplots(figsize=(8, 8))

# Target clusters in background
target = model.sample_target(300)
ax.scatter(target[:, 0], target[:, 1], alpha=0.2, s=10, c='red', label='Target clusters')

# Plot trajectories
n_traj = 30
colors = plt.cm.viridis(np.linspace(0, 1, n_traj))
for i in range(n_traj):
    traj = trajectories[:, i, :]
    ax.plot(traj[:, 0], traj[:, 1], c=colors[i], alpha=0.6, linewidth=0.8)
    ax.scatter(traj[0, 0], traj[0, 1], c='blue', s=30, marker='o', zorder=5)  # Start
    ax.scatter(traj[-1, 0], traj[-1, 1], c='green', s=30, marker='s', zorder=5)  # End

ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
ax.set_title('Flow Trajectories: Noise (blue) → Clusters (green)', fontsize=14)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend(['Target', 'Trajectory', 'Start (t=0)', 'End (t=1)'])
ax.grid(True, alpha=0.3)
plt.show()

print("\n→ Blue dots: Starting points (random Gaussian noise)")
print("→ Green squares: Ending points (clustered around targets)")
print("→ Lines: The path each point follows guided by velocity field")

---
# Loom video recording 2: ODE Solvers Comparison (2 min)

**Key points to explain:**
1. Euler: Simplest, 1st order, 1 function eval/step (fastest but least accurate)
2. RK4: 4th order, 4 function evals/step (most accurate but slowest)
3. Trade-off: Speed vs Accuracy

---

In [ ]:
# 🔊 AUDIO ODE SOLVER COMPARISON - Loom video recording 2
print("AUDIO: Comparing ODE Solvers (Euler vs RK4)")
print("=" * 50)

import time

np.random.seed(42)
noise = generate_noise(duration=1.0)
tone = generate_target_tone(frequency=440, duration=1.0)

# ============================================
# STIFF ODE - Euler fails dramatically here!
# ============================================
class StiffAudioFlow:
    """Stiff ODE where Euler becomes unstable but RK4 handles it."""
    
    def __init__(self, target_audio, stiffness=15.0):
        self.target = target_audio
        self.stiffness = stiffness
    
    def get_velocity(self, x, t):
        """v = stiffness * (target - x)"""
        return self.stiffness * (self.target - x)

def generate_stiff_flow_stepwise(source, target, step_fn, num_steps, stiffness=15.0):
    """Returns intermediate results at each step."""
    model = StiffAudioFlow(target, stiffness)
    dt = 1.0 / num_steps
    x = source.copy()
    
    steps_results = [source.copy()]
    for i in range(num_steps):
        t = i * dt
        x = step_fn(x, t, dt, model.get_velocity)
        steps_results.append(x.copy())
    
    return steps_results

def generate_stiff_flow_timed(source, target, step_fn, num_steps, stiffness=15.0):
    """Returns final result and execution time."""
    model = StiffAudioFlow(target, stiffness)
    dt = 1.0 / num_steps
    x = source.copy()
    
    start = time.perf_counter()
    for i in range(num_steps):
        t = i * dt
        x = step_fn(x, t, dt, model.get_velocity)
    elapsed = time.perf_counter() - start
    
    return x, elapsed

# ============================================
# SPEED vs ACCURACY TRADE-OFF
# ============================================
print("\n" + "=" * 50)
print("SPEED vs ACCURACY TRADE-OFF")
print("=" * 50)
print("\nEuler: 1 velocity eval/step (FAST)")
print("RK4:   4 velocity evals/step (SLOW but ACCURATE)\n")

# Time both solvers with same steps
stiffness = 15.0  # Lower stiffness for fair comparison
test_steps = [10, 50, 100, 500]

print(f"{'Steps':<8} {'Euler Time':<15} {'RK4 Time':<15} {'RK4/Euler':<10}")
print("-" * 50)

for steps in test_steps:
    # Run multiple times for accurate timing
    n_runs = 10
    
    euler_times = []
    rk4_times = []
    for _ in range(n_runs):
        _, t_euler = generate_stiff_flow_timed(noise, tone, audio_euler_step, steps, stiffness)
        _, t_rk4 = generate_stiff_flow_timed(noise, tone, audio_rk4_step, steps, stiffness)
        euler_times.append(t_euler)
        rk4_times.append(t_rk4)
    
    avg_euler = np.mean(euler_times) * 1000  # ms
    avg_rk4 = np.mean(rk4_times) * 1000
    ratio = avg_rk4 / avg_euler
    
    print(f"{steps:<8} {avg_euler:>8.2f} ms     {avg_rk4:>8.2f} ms     {ratio:.1f}x slower")

print("\n→ RK4 is ~4x slower (4 velocity evals vs 1)")
print("→ BUT: RK4 needs fewer steps for same accuracy!")

# ============================================
# DEMONSTRATE: RK4 with fewer steps = Euler with more steps
# ============================================
print("\n" + "=" * 50)
print("ACCURACY COMPARISON (same computation time)")
print("=" * 50)

# RK4 with 10 steps vs Euler with 40 steps (same # of velocity evals)
stiffness = 10.0  # Stable for both

rk4_10, time_rk4 = generate_stiff_flow_timed(noise, tone, audio_rk4_step, 10, stiffness)
euler_40, time_euler = generate_stiff_flow_timed(noise, tone, audio_euler_step, 40, stiffness)

# Error from target
rk4_error = np.mean(np.abs(rk4_10 - tone))
euler_error = np.mean(np.abs(euler_40 - tone))

print(f"\nRK4 (10 steps, 40 evals):   error = {rk4_error:.4f}, time = {time_rk4*1000:.2f}ms")
print(f"Euler (40 steps, 40 evals): error = {euler_error:.4f}, time = {time_euler*1000:.2f}ms")
print(f"\n→ Same computation cost, RK4 error is {euler_error/rk4_error:.1f}x smaller!")

print("\n▶ RK4 (10 steps):")
play(np.clip(rk4_10, -1, 1))
print("\n▶ Euler (40 steps):")
play(np.clip(euler_40, -1, 1))

# ============================================
# STEP-WISE VISUALIZATION (Stiff ODE)
# ============================================
print("\n" + "=" * 50)
print("STEP-BY-STEP: EULER INSTABILITY")
print("=" * 50)

num_steps = 8
stiffness = 20.0

print(f"\nStiff ODE: stiffness={stiffness}, {num_steps} steps")
print(f"Euler stability: |1 - stiffness*dt| = |1 - {stiffness/num_steps:.2f}| = {abs(1-stiffness/num_steps):.2f} > 1 → UNSTABLE!")

euler_steps = generate_stiff_flow_stepwise(noise, tone, audio_euler_step, num_steps, stiffness)
rk4_steps = generate_stiff_flow_stepwise(noise, tone, audio_rk4_step, num_steps, stiffness)

for step in [0, 4, 8]:
    print(f"\n--- Step {step}/{num_steps} ---")
    euler_max = np.max(np.abs(euler_steps[step]))
    rk4_max = np.max(np.abs(rk4_steps[step]))
    
    print(f"Euler max: {euler_max:.2f} | RK4 max: {rk4_max:.2f}")
    print(f"▶ Euler:")
    play(np.clip(euler_steps[step], -1, 1))
    print(f"▶ RK4:")
    play(np.clip(rk4_steps[step], -1, 1))

# Plot
fig, axes = plt.subplots(2, 5, figsize=(18, 6))
steps_to_show = [0, 2, 4, 6, 8]

for idx, step in enumerate(steps_to_show):
    axes[0, idx].plot(euler_steps[step][:300], 'g', linewidth=0.5)
    axes[0, idx].set_title(f"Step {step}\nmax={np.max(np.abs(euler_steps[step])):.1f}")
    axes[0, idx].set_ylim(-3, 3)
    axes[0, idx].grid(True, alpha=0.3)
    
    axes[1, idx].plot(rk4_steps[step][:300], 'orange', linewidth=0.5)
    axes[1, idx].set_title(f"Step {step}\nmax={np.max(np.abs(rk4_steps[step])):.2f}")
    axes[1, idx].set_ylim(-0.5, 0.5)
    axes[1, idx].grid(True, alpha=0.3)

axes[0, 0].set_ylabel("EULER\n(unstable)", fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel("RK4\n(stable)", fontsize=12, fontweight='bold')
plt.suptitle("Step-by-Step: Euler Blows Up, RK4 Stays Stable", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + "=" * 50)
print("SUMMARY: SPEED vs ACCURACY TRADE-OFF")
print("=" * 50)
print("• Euler: 1 eval/step → FAST but less accurate")
print("• RK4:   4 evals/step → 4x SLOWER but much more accurate")
print("• For same accuracy: RK4 needs ~4x fewer steps")
print("• For stiff ODEs: Euler becomes UNSTABLE, RK4 stays stable")

In [ ]:
# 2.2 Show solver comparison plot
print("\nGenerating samples with each solver (30 steps)...")
print("-" * 50)

solvers = {"Euler": euler_step, "RK4": rk4_step}
num_steps = 30
n_samples = 200

results = {}
for name, solver in solvers.items():
    model.rng = np.random.default_rng(42)
    samples = generate_samples(model, solver, n_samples=n_samples, num_steps=num_steps)
    results[name] = samples
    print(f"{name}: Mean=[{samples[:, 0].mean():+.3f}, {samples[:, 1].mean():+.3f}], Std=[{samples[:, 0].std():.3f}, {samples[:, 1].std():.3f}]")

# Plot comparison
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Source
model.rng = np.random.default_rng(42)
source = model.sample_source(200)
axes[0].scatter(source[:, 0], source[:, 1], alpha=0.5, s=15, c='blue')
axes[0].set_title("Source\n(Gaussian Noise)", fontsize=12)

# Each solver
for idx, (name, samples) in enumerate(results.items()):
    axes[idx+1].scatter(samples[:, 0], samples[:, 1], alpha=0.5, s=15, c='green')
    axes[idx+1].set_title(f"{name}\n(30 steps)", fontsize=12)

# Target
model.rng = np.random.default_rng(42)
target = model.sample_target(200)
axes[3].scatter(target[:, 0], target[:, 1], alpha=0.5, s=15, c='red')
axes[3].scatter(model.centers[:, 0], model.centers[:, 1], c='darkred', s=100, marker='x', linewidths=2)
axes[3].set_title("Target\n(4 Clusters)", fontsize=12)

for ax in axes:
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle("ODE Solver Comparison (30 steps each)", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 2.3 Show effect of num_steps on Euler (to demonstrate why accuracy matters)
print("\nEuler with different step counts:")
print("-" * 50)

for steps in [10, 50]:
    model.rng = np.random.default_rng(42)
    samples = generate_samples(model, euler_step, n_samples=150, num_steps=steps)
    distances = [np.min(np.linalg.norm(model.centers - s, axis=1)) for s in samples]
    print(f"  {steps} steps: Avg distance to cluster = {np.mean(distances):.3f}")

print("\n→ More steps = better accuracy (but slower)")
print("→ Better solver (RK4) can achieve same accuracy with fewer steps")